In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
bids = pd.read_csv("/kaggle/input/facebook-recruiting-iv-human-or-bot/bids.csv.zip") #크기가 아주 크기떄문에 run all 없이 한다. + 따로 배치+ 주석처리

In [ ]:
from datetime import datetime
bids["datetime"]=bids["time"].apply(lambda x:datetime.fromtimestamp(x / 7000000))

In [ ]:
bids["year"]=bids["datetime"].dt.year
bids["month"]=bids["datetime"].dt.month
bids["day"]=bids["datetime"].dt.day
bids["hour"] = bids["datetime"].dt.hour

In [ ]:
time_gap=bids.groupby("bidder_id")["time"].apply(lambda x: (x.max() - x.min()))
time_gap

In [ ]:
date=bids.groupby("bidder_id")["year","month","day","hour"].mean().reset_index()

In [ ]:
nunique=bids.groupby("bidder_id")["auction","device","country"].nunique().reset_index() #reset_index() 데이터 프레임으로 바꿈.
# to_frame("auction_nunique") 데이터프레임으로 바꾸고 이름 바꿈
#rename도 사용 ex)  rename({"auction":"auction_nunique","device":"device_nunique","country":"country_nunique"})

In [ ]:
nunique

In [ ]:
id_count = bids.groupby("bidder_id")["ip"].count().reset_index()

In [ ]:
train = pd.read_csv("/kaggle/input/facebook-recruiting-iv-human-or-bot/train.csv.zip")
test = pd.read_csv("/kaggle/input/facebook-recruiting-iv-human-or-bot/test.csv.zip")

display(train, test)

In [ ]:
all_data = pd.concat([train,test])
all_data.nunique() #데이터중 안 겹치는 것의 개수

In [ ]:
all_data = pd.merge(all_data,time_gap,on="bidder_id",how="left")

In [ ]:
all_data = pd.merge(all_data,id_count,on="bidder_id",how="left")

In [ ]:
all_data = pd.merge(all_data,date,on="bidder_id",how='left')

In [ ]:
all_data = pd.merge(all_data,nunique,on="bidder_id",how="left")
all_data

In [ ]:
all_data["time"] = all_data["time"] / all_data["ip"]

In [ ]:
all_data["auction_ratio"] = all_data["ip"] / all_data["auction"]
all_data["device_ratio"] = all_data["ip"] / all_data["device"]

In [ ]:
all_data.sort_values("address").head(50)

In [ ]:
all_data["address"] = all_data["address"].apply(lambda x:x[:5])
all_data["address"]

In [ ]:
all_data["payment_account"] = all_data["payment_account"].apply(lambda x:x[:5])
all_data["payment_account"]


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
all_data["address"]=le.fit_transform(all_data["address"])
all_data["payment_account"]=le.fit_transform(all_data["payment_account"])

all_data2 = all_data.drop(["outcome","bidder_id"], axis=1)
all_data2=  all_data2.fillna(-1)

In [ ]:
train2 = all_data2[:len(train)]
test2 = all_data2[len(train):]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(20,12))
sns.boxplot(all_data["outcome"],all_data["time"], showfliers=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(20,12))
sns.boxplot(all_data["outcome"],all_data["auction_ratio"], showfliers=False)

In [ ]:
# plt.figure(figsize=(20,12))
# sns.boxplot(all_data["outcome"],all_data["id_count"], showfliers=False)

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# plt.figure(figsize=(12,20))
# sns.countplot(train["outcome"])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
rf.fit(train2,train["outcome"])
result = rf.predict_proba(test2)[:,1]

In [ ]:
submission = pd.read_csv("/kaggle/input/facebook-recruiting-iv-human-or-bot/sampleSubmission.csv")
submission["prediction"] = result
submission.to_csv("sub_face_robot.csv" ,index=0)